<figure>
<center>
<img src="../Imagenes/logo_final.png"  align="left"/> 
</center>   
</figure>

# <span style="color:red"><center>BERT-Fine-tuning</center></span>

<center>Ajuste fino de modelos pre-entrenados </center>

##   <span style="color:blue">Autores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Diseño gráfico y Marketing digital</span>
 

1. Maria del Pilar Montenegro Reyes, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

## <span style="color:blue">Referencias</span> 

1. [Zero-shot learning](https://en.wikipedia.org/wiki/Zero-shot_learning)
1. [HuggingFace BERT model](https://huggingface.co/transformers/model_doc/bert.html)
1. [Getting Started with Google BERT: Build and train state-of-the-art natural language processing models using BERT](http://library.lol/main/A0CA3A1276D07957FD7B28F843C299BA)
1. [Transformers for Natural Language Processing: Build innovative deep neural network architectures for NLP with Python, PyTorch, TensorFlow, BERT, RoBERTa, and more](http://library.lol/main/A8C97E552646B3F194ECA333221CEE88)
1. [HuggingFace. Transformers ](https://huggingface.co/transformers/)
1. [HuggingFace. Intro pipeline](https://huggingface.co/course/chapter1/3?fw=pt)
1. [Tutorial Transformer de Google](https://www.tensorflow.org/text/tutorials/transformer)
1. [Transformer-chatbot-tutorial-with-tensorflow-2](https://blog.tensorflow.org/2019/05/transformer-chatbot-tutorial-with-tensorflow-2.html) 
1. [Transformer Architecture: The positional encoding](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/)
1. [Illustrated Auto-attención](https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a)
1. [Illustrated Attention](https://towardsdatascience.com/attn-illustrated-attention-5ec4ad276ee3#0458)
1. [Neural Machine Translation by Jointly Learning to Align and Translate (Bahdanau et. al, 2015)](https://arxiv.org/pdf/1409.0473.pdf)
1. [Effective Approaches to Attention-based Neural Machine Translation (Luong et. al, 2015)](https://arxiv.org/pdf/1508.04025.pdf)
1. [Attention Is All You Need (Vaswani et. al, 2017)](https://arxiv.org/pdf/1706.03762.pdf)
1. [Self-Attention GAN (Zhang et. al, 2018)](https://arxiv.org/pdf/1805.08318.pdf)
1. [Sequence to Sequence Learning with Neural Networks (Sutskever et. al, 2014)](https://arxiv.org/pdf/1409.3215.pdf)
1. [TensorFlow’s seq2seq Tutorial with Attention (Tutorial on seq2seq+attention)](https://github.com/tensorflow/nmt)
1. [Lilian Weng’s Blog on Attention (Great start to attention)](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html#a-family-of-attention-mechanisms)
1. [Jay Alammar’s Blog on Seq2Seq with Attention (Great illustrations and worked example on seq2seq+attention)](https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/)
1. [Google’s Neural Machine Translation System: Bridging the Gap between Human and Machine Translation (Wu et. al, 2016)](https://arxiv.org/pdf/1609.08144.pdf)
1. [Adam: A method for stochastic optimization](https://arxiv.org/pdf/1412.6980.pdf)
1.[Stanford IMBS sentiment analysis data base](http://ai.stanford.edu/~amaas/data/sentiment/)

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Clasificación de textos](#Clasificación-de-textos)

## <span style="color:blue">Introducción</span>

Usaremos la implementación de HuggingFace in en Pytorch. 

En esta lección revisamo como hacer el ajuste-fino de modelos preentrenados para la siguientes tareas de lenguaje natural.

* Clasificación de textos.
* Inferencia de lenguaje natural.
* Reconocimiento de nombres de entidades. NER.
* Pregunta-Respuesta

## <span style="color:blue">Clasificación de textos</span>

La tarea de PLN es análisis de sentimiento. El primer experimento lo hacemos en inglés y luego en Español. Para esta tarea esta bién usar el modelo uncase (eliminando mayúsculas). 

Los datos corresponde a [Stanford IMBS sentiment analysis data base](http://ai.stanford.edu/~amaas/data/sentiment/). Son 25K comentarios para entrenamiento y 25K para prueba. Los datos fueron obtenidos de [Kaggle](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/version/1).

### Lectura de datos

In [41]:
import pandas as pd

df= pd.read_csv('../Datos/IMDB/IMDB Dataset.csv', sep=',', encoding='utf-8')

#df["sentiment"] = df["sentiment"].astype('category')

df.rename(columns = {'review': 'text', 'sentiment':'label'}, inplace=True)

df.head()

,text,label
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [42]:
df.dtypes

text     object
label    object
dtype: object

In [34]:
print(df.shape)
print(df.dtypes)


(50000, 2)
text       object
label    category
dtype: object


In [39]:
df.shape

(50000, 2)

#### Construyendo nlp.Dataset a partir de Pandas dataframe

In [43]:
# Separa los datos en entranamiento y validación
import numpy as np

# inicializa el generador de pseudo-aleatorios
#np.random.RandomState(2021)
np.random.seed(2021)
size = df.shape[0]

train_size = int(size*0.7)
test_size = size - train_size

train_index = np.random.choice(np.arange(size), size=train_size, replace=False)
test_index = np.array(list(set(np.arange(size)) - set(train_index)))

# separa los conjuntos de datos
#train_dataset = df.loc[train_index]
#test_dataset = df.loc[test_idx]
#train_dataset.reset_index(drop=True, inplace=True)

# crea el npl.Dataset
from nlp import Dataset
train_ds = Dataset.from_pandas(df.loc[train_index])
test_ds = Dataset.from_pandas(df.loc[test_index])

In [44]:
#train_dataset
train_ds

Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None)}, num_rows: 35000)

Alternativamente podemos leer todos los datos y subdividirlos en el datses así:

In [ ]:
dset = Dataset.from_pandas(df)
#dset = dset.train_test_split(test_size=0.3)# por defecto shuffle=True

In [47]:
help(dset)

Help on DatasetDict in module nlp.dataset_dict object:

class DatasetDict(builtins.dict)
 |  A dictionary (dict of str: nlp.Dataset) with dataset transforms methods (map, filter, etc.)
 |  
 |  Method resolution order:
 |      DatasetDict
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  cast_(self, features: nlp.features.Features)
 |      Cast the dataset to a new set of features.
 |      The transformation is applied to all the datasets of the dataset dictionary.
 |      
 |      You can also remove a column using :func:`Dataset.map` with `feature` but :func:`cast_`
 |      is in-place (doesn't copy the data to a new dataset) and is thus faster.
 |      
 |      Args:
 |          features (:class:`nlp.Features`): New features to cast the dataset to.
 |              The name and order of the fields in the features must match the current column names.
 |              The type of the data must also be convertible from one type to the other.
 |        

In [23]:
dset

{'train': Dataset(features: {'review': Value(dtype='string', id=None), 'sentiment': Value(dtype='string', id=None)}, num_rows: 35000),
 'test': Dataset(features: {'review': Value(dtype='string', id=None), 'sentiment': Value(dtype='string', id=None)}, num_rows: 15000)}

#### Carga los datos desde HuggingFace Hub

In [ ]:
# lista de datasets
from nlp import list_datasets
datasets_list = list_datasets()

print(len(datasets_list))
print(', '.join(dataset.id for dataset in datasets_list))



Encontramos dos datasets: imdb, imdb_urdu_reviews. Vamos a explorarlos:

* [imdb dataset](https://huggingface.co/datasets/viewer/?dataset=imdb)
* [imdb_urdu_reviews dataset](https://huggingface.co/datasets/viewer/?dataset=imdb_urdu_reviews). Traducción de imdb al lenguaje urdu.

Usaremos de momento el dataset `imdb`.

In [16]:
from nlp import load_dataset
dataset = load_dataset('imdb')

In [17]:
print(type(dataset))
print(dataset)

<class 'nlp.dataset_dict.DatasetDict'>
{'train': Dataset(features: {'text': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['neg', 'pos'], names_file=None, id=None)}, num_rows: 25000), 'test': Dataset(features: {'text': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['neg', 'pos'], names_file=None, id=None)}, num_rows: 25000), 'unsupervised': Dataset(features: {'text': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['neg', 'pos'], names_file=None, id=None)}, num_rows: 50000)}
